In [1]:
import pandas as pd
import numpy as np

In [2]:
train=pd.read_excel("D:\EVUEME\MAY\MAY_18\Train_Feature.xlsx")
val=pd.read_excel("D:\EVUEME\MAY\MAY_18\Val_Feature.xlsx")
test=pd.read_excel("D:\EVUEME\MAY\MAY_18\Test_Feature.xlsx")

In [3]:
train_video_features = train.iloc[:,10:721]
train_audio_features = train.iloc[:,721:1404]
train_text_features = train.iloc[:,1404:]
train_y=train.iloc[:,4:10]

In [4]:
val_video_features=val.iloc[:,10:721]
val_audio_features=val.iloc[:,721:1404]
val_text_features=val.iloc[:,1404:]
val_y=val.iloc[:,4:10]

In [5]:
test_video_features= test.iloc[:,10:721]
test_audio_features=test.iloc[:,721:1404]
test_text_features=test.iloc[:,1404:]
test_y=test.iloc[:,4:10]

In [6]:
audio_r,audio_feat=train_audio_features.shape
audio_feat

video_r,video_feat=train_video_features.shape
video_feat

text_r,text_feat=train_text_features.shape
text_feat

op_r,op=train_y.shape
op

6

In [7]:
import tensorflow as tf
from tensorflow.keras import Model,Sequential,initializers
from tensorflow.keras.layers import Dense,Dropout
from sklearn.metrics import mean_absolute_error
from tensorflow.keras.layers import Concatenate
from sklearn.preprocessing import StandardScaler
tf.random.set_seed(42)## taking the seed as 42 
initializer=tf.keras.initializers.GlorotUniform()

In [8]:
## for the audio model 

audio_input=tf.keras.Input(shape=(audio_feat),name="audio_input")
audio_subnetwork=Dense(64)(audio_input)

## for the video 

video_input=tf.keras.Input(shape=(video_feat),name="video_input")
video_subnetwork=Dense(64)(video_input)

## for the text 

text_input=tf.keras.Input(shape=(text_feat),name="text_input")
text_subnetwork=Dense(64)(text_input)

In [9]:
merged=Concatenate()([audio_subnetwork,video_subnetwork,text_subnetwork])

In [10]:
merged=Dense(512,activation='relu',kernel_initializer=initializer)(merged)
merged=Dropout(0.2)(merged)

merged=Dense(256,activation="relu",kernel_initializer=initializer)(merged)
merged=Dropout(0.2)(merged)

merged=Dense(128,activation="relu",kernel_initializer=initializer)(merged)
merged=Dropout(0.2)(merged)

merged=Dense(64,activation="relu",kernel_initializer=initializer)(merged)
merged=Dropout(0.2)(merged)

merged=Dense(32,activation="relu",kernel_initializer=initializer)(merged)
merged=Dropout(0.2)(merged)

output_model=Dense(op,activation="sigmoid",kernel_initializer=initializer)(merged)

c:\Python3.9.4inst\lib\site-packages\keras\initializers\initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [11]:
final_model=Model(inputs=[audio_input,video_input,text_input],outputs=output_model)

In [12]:
final_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 audio_input (InputLayer)       [(None, 683)]        0           []                               
                                                                                                  
 video_input (InputLayer)       [(None, 711)]        0           []                               
                                                                                                  
 text_input (InputLayer)        [(None, 66)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           43776       ['audio_input[0][0]']            
                                                                                              

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
ES=EarlyStopping(patience=10)

In [14]:
from tensorflow.keras.optimizers import Adam
optimizer=Adam()
final_model.compile(loss="mean_absolute_error",optimizer=optimizer)

In [15]:
def scale_datasets(x_train, x_val, x_test):
      """
      Standard Scale test and train data
      Z - Score normalization
      """
      standard_scaler = StandardScaler()
      x_train_scaled = pd.DataFrame(
          standard_scaler.fit_transform(x_train),
          columns=x_train.columns
      )
      x_val_scaled = pd.DataFrame(
          standard_scaler.transform(x_val),
          columns = x_test.columns
      )
      x_test_scaled = pd.DataFrame(
          standard_scaler.transform(x_test),
          columns = x_test.columns
      )
      return x_train_scaled,x_val_scaled, x_test_scaled


In [16]:
train_video_scaled, val_video_scaled, test_video_scaled = scale_datasets(train_video_features, val_video_features ,test_video_features)
train_audio_scaled, val_audio_scaled, test_audio_scaled = scale_datasets(train_audio_features, val_audio_features ,test_audio_features)
train_text_scaled, val_text_scaled, test_text_scaled = scale_datasets(train_text_features, val_text_features ,test_text_features)

In [17]:
history=final_model.fit(
    x=[train_audio_scaled.values,train_video_scaled.values,train_text_scaled.values],
    y=train_y.values,
    validation_data=([val_audio_scaled.values,val_video_scaled.values,val_text_scaled.values],val_y.values),
    epochs=50,   
    callbacks=ES
)

Epoch 1/50
186/186 [==============================] - 3s 6ms/step - loss: 0.1179 - val_loss: 0.1078
Epoch 2/50
186/186 [==============================] - 1s 6ms/step - loss: 0.1075 - val_loss: 0.1068
Epoch 3/50
186/186 [==============================] - 1s 5ms/step - loss: 0.1030 - val_loss: 0.1018
Epoch 4/50
186/186 [==============================] - 1s 5ms/step - loss: 0.1008 - val_loss: 0.0987
Epoch 5/50
186/186 [==============================] - 1s 5ms/step - loss: 0.0990 - val_loss: 0.0995
Epoch 6/50
186/186 [==============================] - 1s 5ms/step - loss: 0.0985 - val_loss: 0.1019
Epoch 7/50
186/186 [==============================] - 1s 5ms/step - loss: 0.0967 - val_loss: 0.1005
Epoch 8/50
186/186 [==============================] - 1s 5ms/step - loss: 0.0959 - val_loss: 0.0987
Epoch 9/50
186/186 [==============================] - 1s 5ms/step - loss: 0.0946 - val_loss: 0.1016
Epoch 10/50
186/186 [==============================] - 1s 5ms/step - loss: 0.0937 - val_loss: 0.1016

In [18]:
y_pred=final_model.predict([train_audio_scaled.values,train_video_scaled.values,train_text_scaled.values])

186/186 [==============================] - 0s 2ms/step


In [19]:
mae=mean_absolute_error(train_y.values,y_pred)

In [20]:
big_six=["extraverasion","neurocitism","agreeableness","conscientoiusness","openess","interview"]
for i in range(6):
    print(big_six[i])
    mae=mean_absolute_error(train_y.values[:,i],y_pred[:,i])
    one_mae=1-mae
    print("MAE",mae,"\n","1-MAE",one_mae,"\n")

extraverasion
MAE 0.0827029239548843 
 1-MAE 0.9172970760451157 

neurocitism
MAE 0.0813742857603202 
 1-MAE 0.9186257142396798 

agreeableness
MAE 0.08354553642194226 
 1-MAE 0.9164544635780577 

conscientoiusness
MAE 0.09330664247942545 
 1-MAE 0.9066933575205746 

openess
MAE 0.083196969573656 
 1-MAE 0.916803030426344 

interview
MAE 0.07914737054286376 
 1-MAE 0.9208526294571362 



validation

In [21]:
y_pred_validation=final_model.predict([val_audio_scaled.values,val_video_scaled.values,val_text_scaled.values])
mae=mean_absolute_error(val_y.values,y_pred_validation)
mae

63/63 [==============================] - 0s 2ms/step


0.09942523867332698

In [22]:
big_six=["extraverasion","neurocitism","agreeableness","conscientoiusness","openess","interview"]
for i in range(6):
    print(big_six[i])
    mae=mean_absolute_error(val_y.values[:,i],y_pred_validation[:,i])
    one_mae=1-mae
    print("MAE",mae,"\n","1-MAE",one_mae,"\n")

extraverasion
MAE 0.09846114991931779 
 1-MAE 0.9015388500806822 

neurocitism
MAE 0.1018157776968883 
 1-MAE 0.8981842223031117 

agreeableness
MAE 0.09154704195504884 
 1-MAE 0.9084529580449512 

conscientoiusness
MAE 0.109336116583476 
 1-MAE 0.890663883416524 

openess
MAE 0.09824761979727789 
 1-MAE 0.9017523802027221 

interview
MAE 0.09714372608795334 
 1-MAE 0.9028562739120467 



test

In [23]:
y_pred_test=final_model.predict([test_audio_scaled.values,test_video_scaled.values,test_text_scaled.values])
mae=mean_absolute_error(test_y.values,y_pred_test)
mae

63/63 [==============================] - 0s 2ms/step


0.10030068270117037

In [24]:
big_six=["extraverasion","neurocitism","agreeableness","conscientoiusness","openess","interview"]
for i in range(6):
    print(big_six[i])
    mae=mean_absolute_error(test_y.values[:,i],y_pred_test[:,i])
    one_mae=1-mae
    print("MAE",mae,"\n","1-MAE",one_mae,"\n")

extraverasion
MAE 0.0994608104034507 
 1-MAE 0.9005391895965493 

neurocitism
MAE 0.10164838937826862 
 1-MAE 0.8983516106217314 

agreeableness
MAE 0.09543551254816848 
 1-MAE 0.9045644874518315 

conscientoiusness
MAE 0.10799166524439617 
 1-MAE 0.8920083347556038 

openess
MAE 0.09960609436194258 
 1-MAE 0.9003939056380574 

interview
MAE 0.09766162427079583 
 1-MAE 0.9023383757292042 



classification task 

In [27]:
import numpy as np
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)  
cluster_labels = kmeans.fit_predict(train_y)

thresholds = []
for i in range(kmeans.n_clusters):
    cluster_data = train_y[cluster_labels == i]
    cluster_thresholds = np.percentile(cluster_data, [ 25,50, 75,100], axis=0) 
    thresholds.append(('Cluster {}'.format(i), cluster_thresholds))

for cluster, cluster_thresholds in thresholds:
    print(cluster)
    print('Low Thresholds:', cluster_thresholds[0])
    print('Mid Thresholds:', cluster_thresholds[1])
    print('High Thresholds:', cluster_thresholds[2])
    print()


Cluster 0
Low Thresholds: [0.39252336 0.44791667 0.48351648 0.4368932  0.48888889 0.43925234]
Mid Thresholds: [0.45794393 0.51041667 0.53846154 0.50485437 0.55555556 0.48598131]
High Thresholds: [0.51401869 0.5625     0.59340659 0.58252427 0.61111111 0.53271028]

Cluster 1
Low Thresholds: [0.56074766 0.61458333 0.61538462 0.58252427 0.64444444 0.60747664]
Mid Thresholds: [0.62616822 0.66666667 0.67032967 0.66019417 0.7        0.64953271]
High Thresholds: [0.68224299 0.72916667 0.72527473 0.72815534 0.76666667 0.71028037]

Cluster 2
Low Thresholds: [0.23364486 0.26041667 0.31868132 0.27184466 0.32222222 0.25233645]
Mid Thresholds: [0.29906542 0.32291667 0.38461538 0.33980583 0.38888889 0.30841121]
High Thresholds: [0.35514019 0.38541667 0.45054945 0.40776699 0.45555556 0.35514019]



c:\Python3.9.4inst\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
